# Battle of Neighborhoods - New York

### Import all libraries

In [238]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [239]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [240]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [241]:
neighborhoods_data = newyork_data['features']

In [242]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [243]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [244]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [245]:
for data in neighborhoods_data:
    borough = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [246]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


### There must be 5 boroughs and 306 neighborhoods in New York. 

In [247]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [248]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### New York neighborhoods can be visualized in below map

In [40]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [249]:
neighborhoods.Borough.unique()

array(['Bronx', 'Manhattan', 'Brooklyn', 'Queens', 'Staten Island'],
      dtype=object)

### Now analyze data for Brooklyn borough

In [250]:
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [251]:
address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of "{}" are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of "Brooklyn, NY" are 40.6501038, -73.9495823.


In [252]:
# create map of Brooklyn using latitude and longitude values
map_brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

### Fetch venue details for Brooklyn from Foursquare API

In [254]:
# @hidden_cell
CLIENT_ID = 'WJXUZZWGI03AOG15CSOU2LOXF41XY1JK4SKBDJYUJ0GABZT0' # your Foursquare ID
CLIENT_SECRET = 'R2HFNFDVE30JO2A15ZVJRSV4C30N0SWPFJAKYODU23WQF10J' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WJXUZZWGI03AOG15CSOU2LOXF41XY1JK4SKBDJYUJ0GABZT0
CLIENT_SECRET:R2HFNFDVE30JO2A15ZVJRSV4C30N0SWPFJAKYODU23WQF10J


In [255]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print("Fetching nearby venues for neighborhood: ", name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [257]:
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )

Fetching nearby venues for neighborhood:  Bay Ridge
Fetching nearby venues for neighborhood:  Bensonhurst
Fetching nearby venues for neighborhood:  Sunset Park
Fetching nearby venues for neighborhood:  Greenpoint
Fetching nearby venues for neighborhood:  Gravesend
Fetching nearby venues for neighborhood:  Brighton Beach
Fetching nearby venues for neighborhood:  Sheepshead Bay
Fetching nearby venues for neighborhood:  Manhattan Terrace
Fetching nearby venues for neighborhood:  Flatbush
Fetching nearby venues for neighborhood:  Crown Heights
Fetching nearby venues for neighborhood:  East Flatbush
Fetching nearby venues for neighborhood:  Kensington
Fetching nearby venues for neighborhood:  Windsor Terrace
Fetching nearby venues for neighborhood:  Prospect Heights
Fetching nearby venues for neighborhood:  Brownsville
Fetching nearby venues for neighborhood:  Williamsburg
Fetching nearby venues for neighborhood:  Bushwick
Fetching nearby venues for neighborhood:  Bedford Stuyvesant
Fetchin

In [258]:
print(brooklyn_venues.shape)
brooklyn_venues.head()

(2761, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Georgian Dream Cafe and Bakery,40.625586,-74.030196,Caucasian Restaurant
3,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.624200,-74.030931,Pizza Place
4,Bay Ridge,40.625801,-74.030621,Cocoa Grinder,40.623967,-74.030863,Juice Bar


In [259]:
brooklyn_venues["Venue Category"].unique()

array(['Spa', 'Bagel Shop', 'Caucasian Restaurant', 'Pizza Place',
       'Juice Bar', 'Breakfast Spot', 'Taco Place', 'Grocery Store',
       'Greek Restaurant', 'Bookstore', 'Middle Eastern Restaurant',
       'Hookah Bar', 'Bar', 'Lounge', 'Optical Shop', 'Sports Bar',
       'Italian Restaurant', 'Tea Room', 'Café', 'American Restaurant',
       'Ice Cream Shop', 'New American Restaurant', 'Video Game Store',
       'Chinese Restaurant', 'Toy / Game Store', 'Sushi Restaurant',
       'Thai Restaurant', 'Department Store', 'Seafood Restaurant',
       'Mexican Restaurant', 'Vietnamese Restaurant', 'Clothing Store',
       'Indian Restaurant', 'Sandwich Place', 'Lingerie Store',
       'Pool Hall', 'Dim Sum Restaurant', 'Pharmacy', 'Snack Place',
       'Kids Store', 'Fast Food Restaurant', 'Gym / Fitness Center',
       'Mattress Store', 'Supermarket', 'Hotel', 'Yoga Studio',
       'Boutique', 'Mediterranean Restaurant', 'Electronics Store',
       'Playground', 'Park', 'Cosmetics 

### Filter out all restaurants from available venues

In [260]:
brooklyn_restaurants = brooklyn_venues[brooklyn_venues["Venue Category"].str.contains("Restaurant|Diner|Taverna|Steakhouse")]
print(brooklyn_restaurants.shape)
brooklyn_restaurants.head()

(668, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2,Bay Ridge,40.625801,-74.030621,Georgian Dream Cafe and Bakery,40.625586,-74.030196,Caucasian Restaurant
8,Bay Ridge,40.625801,-74.030621,Blue Door Souvlakia,40.624567,-74.030311,Greek Restaurant
11,Bay Ridge,40.625801,-74.030621,Karam,40.622931,-74.028316,Middle Eastern Restaurant
19,Bay Ridge,40.625801,-74.030621,Elia Restaurant,40.623090,-74.031156,Greek Restaurant
20,Bay Ridge,40.625801,-74.030621,Tuscany Grill,40.622913,-74.031387,Italian Restaurant


### So, we have 668 restaurants in Brooklyn borough.

### Encode restaurants data to find out where Italian restaurants are more in number.

In [261]:
# one hot encoding
brooklyn_onehot = pd.get_dummies(brooklyn_restaurants[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = brooklyn_restaurants['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_onehot.head()

,Neighborhood,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Burmese Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Cuban Restaurant,Dim Sum Restaurant,Diner,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Hotpot Restaurant,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,Pakistani Restaurant,Peruvian Restaurant,Polish Restaurant,Ramen Restaurant,Restaurant,Russian Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
2,Bay Ridge,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Bay Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11,Bay Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19,Bay Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20,Bay Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [262]:
brooklyn_onehot.shape

(668, 61)

In [263]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
#brooklyn_grouped.head()
brooklyn_grouped_italian = brooklyn_grouped[["Neighborhood", "Italian Restaurant"]]
brooklyn_grouped_italian.sort_values(by="Italian Restaurant", ascending=False).head(10)

,Neighborhood,Italian Restaurant
39,Madison,0.500000
12,Carroll Gardens,0.458333
34,Gravesend,0.400000
2,Bedford Stuyvesant,0.333333
33,Gowanus,0.266667
31,Georgetown,0.250000
3,Bensonhurst,0.250000
21,Dumbo,0.222222
1,Bay Ridge,0.206897
48,Park Slope,0.187500


### So, we found top 10 neighbourhoods in Brooklyn where you find more Italian restaurants. "Madison" neighborhood tends to show slightly higher average presence of Italian restaurants as compared to other neighborhoods. Are we doing the right thing by choosing "Madison" to dine out? Well, let us try another way to find out best place since we are new to Newyork city. 

### Find top 5 most common restaurants for each neighborhood.

In [264]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [279]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_italian = \
      neighborhoods_venues_sorted[neighborhoods_venues_sorted["1st Most Common Venue"] == "Italian Restaurant"]
neighborhoods_venues_sorted_italian

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Bath Beach,Italian Restaurant,Fast Food Restaurant,Chinese Restaurant,Cantonese Restaurant,Dim Sum Restaurant
1,Bay Ridge,Italian Restaurant,American Restaurant,Greek Restaurant,Thai Restaurant,Sushi Restaurant
8,Brooklyn Heights,Italian Restaurant,Asian Restaurant,Thai Restaurant,Diner,Indian Restaurant
12,Carroll Gardens,Italian Restaurant,Thai Restaurant,French Restaurant,Spanish Restaurant,Latin American Restaurant
14,Clinton Hill,Italian Restaurant,Thai Restaurant,Mexican Restaurant,Japanese Restaurant,Restaurant
15,Cobble Hill,Italian Restaurant,French Restaurant,Thai Restaurant,Mediterranean Restaurant,Middle Eastern Restaurant
21,Dumbo,Italian Restaurant,American Restaurant,Southern / Soul Food Restaurant,Mediterranean Restaurant,Mexican Restaurant
28,Fort Greene,Italian Restaurant,French Restaurant,Vietnamese Restaurant,Restaurant,American Restaurant
29,Fort Hamilton,Italian Restaurant,Chinese Restaurant,Japanese Restaurant,American Restaurant,Diner
33,Gowanus,Italian Restaurant,Chinese Restaurant,Indian Restaurant,Sushi Restaurant,Ramen Restaurant


### Let's map these neighborhoods into clusters to yield more accurate results. We shall use K-Means Clustering algorithm to map neighborhoods into 3 clusters and see which neighborhood/area is more suitable for dining.

In [280]:
# set number of clusters
kclusters = 3

brooklyn_grouped_clustering = brooklyn_grouped_italian.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 2, 2, 2, 2, 0, 2])

In [281]:
#brooklyn_grouped_italian.head()
neighborhoods_venues_sorted_italian.shape

(13, 6)

In [282]:
#neighborhoods_venues_sorted
brooklyn_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,3.0,Italian Restaurant,American Restaurant,Greek Restaurant,Thai Restaurant,Sushi Restaurant
1,Brooklyn,Bensonhurst,40.611009,-73.995180,3.0,Chinese Restaurant,Italian Restaurant,Sushi Restaurant,Hotpot Restaurant,Asian Restaurant
2,Brooklyn,Sunset Park,40.645103,-74.010316,0.0,Latin American Restaurant,Mexican Restaurant,Chinese Restaurant,Italian Restaurant,Vietnamese Restaurant
3,Brooklyn,Greenpoint,40.730201,-73.954241,0.0,French Restaurant,Mexican Restaurant,Restaurant,Sushi Restaurant,Italian Restaurant
4,Brooklyn,Gravesend,40.595260,-73.973471,4.0,Italian Restaurant,Chinese Restaurant,Diner,Eastern European Restaurant,Israeli Restaurant
5,Brooklyn,Brighton Beach,40.576825,-73.965094,2.0,Russian Restaurant,Restaurant,Eastern European Restaurant,Sushi Restaurant,Korean Restaurant
6,Brooklyn,Sheepshead Bay,40.586890,-73.943186,0.0,Turkish Restaurant,Russian Restaurant,Diner,Restaurant,Italian Restaurant
7,Brooklyn,Manhattan Terrace,40.614433,-73.957438,2.0,Chinese Restaurant,Japanese Restaurant,Steakhouse,Dumpling Restaurant,Indian Restaurant
8,Brooklyn,Flatbush,40.636326,-73.958401,2.0,Mexican Restaurant,Caribbean Restaurant,Middle Eastern Restaurant,Vietnamese Restaurant,Eastern European Restaurant
9,Brooklyn,Crown Heights,40.670829,-73.943291,2.0,Kosher Restaurant,Sushi Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Indian Restaurant


In [283]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

brooklyn_merged = brooklyn_data

# merge brooklyn_grouped with brooklyn_data to add latitude/longitude for each neighborhood
brooklyn_merged = brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_merged_italian = brooklyn_merged[brooklyn_merged["1st Most Common Venue"] == "Italian Restaurant"] 
brooklyn_merged_italian = brooklyn_merged_italian.astype({"Cluster Labels": int})
brooklyn_merged_italian

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,0,Italian Restaurant,American Restaurant,Greek Restaurant,Thai Restaurant,Sushi Restaurant
4,Brooklyn,Gravesend,40.595260,-73.973471,1,Italian Restaurant,Chinese Restaurant,Diner,Eastern European Restaurant,Israeli Restaurant
18,Brooklyn,Brooklyn Heights,40.695864,-73.993782,0,Italian Restaurant,Asian Restaurant,Thai Restaurant,Diner,Indian Restaurant
19,Brooklyn,Cobble Hill,40.687920,-73.998561,0,Italian Restaurant,French Restaurant,Thai Restaurant,Mediterranean Restaurant,Middle Eastern Restaurant
20,Brooklyn,Carroll Gardens,40.680540,-73.994654,1,Italian Restaurant,Thai Restaurant,French Restaurant,Spanish Restaurant,Latin American Restaurant
22,Brooklyn,Gowanus,40.673931,-73.994441,0,Italian Restaurant,Chinese Restaurant,Indian Restaurant,Sushi Restaurant,Ramen Restaurant
23,Brooklyn,Fort Greene,40.688527,-73.972906,0,Italian Restaurant,French Restaurant,Vietnamese Restaurant,Restaurant,American Restaurant
24,Brooklyn,Park Slope,40.672321,-73.977050,0,Italian Restaurant,American Restaurant,Mexican Restaurant,Vietnamese Restaurant,Seafood Restaurant
33,Brooklyn,Bath Beach,40.599519,-73.998752,0,Italian Restaurant,Fast Food Restaurant,Chinese Restaurant,Cantonese Restaurant,Dim Sum Restaurant
38,Brooklyn,Clinton Hill,40.693229,-73.967843,0,Italian Restaurant,Thai Restaurant,Mexican Restaurant,Japanese Restaurant,Restaurant


### Visualize clusters in a map for better understanding.

In [294]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brooklyn_merged_italian['Latitude'], brooklyn_merged_italian['Longitude'], \
                                  brooklyn_merged_italian['Neighborhood'], brooklyn_merged_italian['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color="red",
        fill=True,
        fill_color="blue",
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### From above map, it's clearly visible that majority of Italian restaurants are located in north-western part of Brooklyn borough. So, "Madison" may not be the right option to choose as it is located in southern part of Brooklyn. The next option available for us is "Caroll Gardens" or "Bedford Stuyvesant" since we have some more restaurants available in near by neighborhoods.

### We can further fetch tips from Four Square,if available, for each of these restaurants to decide which is the best restaurant.

In [288]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fa7e89b61fa2753f8ba25e2'},
 'response': {'venues': []}}

In [289]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [290]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

In [291]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [292]:
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'

### Well, there are no tips available at the moment for reqwuired restaurants. Hence, let us go to "Caroll Gardens" or "Bedford Stuyvesant" and explore to find out our favuorite Italian restaurant. 